In [23]:
from pathlib import Path

import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
from numpy._typing import NDArray
from numpy.testing import assert_equal
from plotly.graph_objs import Figure
from plotly.io import write_image
from zema_emc_annotated.data_types import SampleSize  # type: ignore[import]
from zema_emc_annotated.dataset import ZeMASamples  # type: ignore[import]

from lp_nn_robustness_verification.data_acquisition.activation_functions import (
    Sigmoid,
)
from lp_nn_robustness_verification.data_acquisition.generate_nn_params import (
    construct_out_features_counts,
    generate_weights_and_biases,
)
from lp_nn_robustness_verification.data_acquisition.uncertain_inputs import (
    UncertainInputs,
)
from lp_nn_robustness_verification.data_types import UncertainArray
from lp_nn_robustness_verification.linear_program import RobustVerifier
from lp_nn_robustness_verification.pre_processing import (
    LinearInclusion,
)

## Preparations

In [24]:
def export_plotly_to_pdf(fig: Figure, filename: str) -> None:
    """Export figure to pdf at the right location right away"""
    PATH_TO_IMAGES = Path(
        "/home/bjorn/code/GUM-compliant_neural_network_"
        "robustness_verification/src/thesis/images"
    )
    write_image(fig, PATH_TO_IMAGES.joinpath(filename))
    print(f"{filename} created")

## Set up shared Plotly layout parameters

In [25]:
pio.templates.default = "plotly"
color_blind_seque = ["#2e2b83", "#7e3e69", "#a56753", "#b7a03c", "#abe600"]
common_font = dict(font=dict(family="Serif", size=15))
common_tlr_margin_for_layout = dict(
    margin_t=15,
    margin_l=63,
    margin_r=63,
)
common_bottom_axis_title_margins = dict(
    margin_b=52,
)
shared_layout = dict(
    **common_font,
    autosize=False,
    showlegend=True,
)
shared_ratio = dict(
    scaleanchor="x",
    scaleratio=1,
)
shared_title_params = dict(
    y=0.9,
    x=0.5,
    xanchor="center",
    yanchor="top",
)

## Run batch of examples with varying random seed

In [26]:
datapoints_per_cycles: list[int] = [1, 10]
depths: list[int] = [1, 3]
for datapoints_per_cycle in datapoints_per_cycles:
    zema_data = ZeMASamples(
        SampleSize(n_cycles=4766, datapoints_per_cycle=datapoints_per_cycle),
        normalize=True,
    )
    for additional_layers in depths:
        print(
            f"Trying to find solution for {datapoints_per_cycle * 11} "
            f"inputs with {additional_layers} layers."
        )
        for idx_sample in range(10):
            uncertain_inputs = UncertainInputs(
                UncertainArray(
                    zema_data.values[idx_sample], zema_data.uncertainties[idx_sample]
                )
            )
            for seed in range(2):
                linear_inclusion = LinearInclusion(
                    uncertain_inputs,
                    Sigmoid,
                    generate_weights_and_biases(
                        len(uncertain_inputs.values),
                        construct_out_features_counts(
                            len(uncertain_inputs.values), depth=additional_layers
                        ),
                        seed,
                    ),
                )
                optimization = RobustVerifier(linear_inclusion)
                optimization.model.hideOutput()
                optimization.solve()
                if optimization.model.getSols():
                    print(
                        f"Objective value for {datapoints_per_cycle * 11} inputs "
                        f"and {additional_layers} layers for sample {idx_sample} "
                        f"with seed {seed}: {optimization.model.getObjVal()}"
                    )

Trying to find solution for 11 inputs with 1 layers.
Objective value for 11 inputs and 1 layers for sample 0 with seed 0: 0.9396282163051026
Objective value for 11 inputs and 1 layers for sample 0 with seed 1: 0.8938419908889208
Objective value for 11 inputs and 1 layers for sample 1 with seed 0: 0.9327868612209433
Objective value for 11 inputs and 1 layers for sample 1 with seed 1: 0.9089630454856501
Objective value for 11 inputs and 1 layers for sample 2 with seed 0: 0.8889323849689708
Objective value for 11 inputs and 1 layers for sample 2 with seed 1: 0.9170373266906816
Objective value for 11 inputs and 1 layers for sample 3 with seed 0: 0.9369728675239944
Objective value for 11 inputs and 1 layers for sample 3 with seed 1: 0.8829350224742357
Objective value for 11 inputs and 1 layers for sample 4 with seed 0: 0.932579444929979
Objective value for 11 inputs and 1 layers for sample 4 with seed 1: 0.841417342165577
Objective value for 11 inputs and 1 layers for sample 5 with seed 0: 

## Solution to the linear optimization problem

In [27]:
zema_data = ZeMASamples(normalize=True)
uncertain_inputs = UncertainInputs(
    UncertainArray(zema_data.values[0][:2], zema_data.uncertainties[0][:2])
)
linear_inclusion = LinearInclusion(
    uncertain_inputs,
    Sigmoid,
    generate_weights_and_biases(
        len(uncertain_inputs.values),
        construct_out_features_counts(len(uncertain_inputs.values), depth=1),
        seed=2,
    ),
)

In [28]:
optimization = RobustVerifier(linear_inclusion)
solution: str = optimization.solve()
print(solution)

Nonepresolving:

(round 1, fast)       2 del vars, 6 del conss, 0 add conss, 4 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (2 rounds: 2 fast, 1 medium, 1 exhaustive):
 6 deleted vars, 6 deleted constraints, 0 added constraints, 4 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
transformed 1/1 original solutions to the transformed problem space
Presolving Time: 0.00

SCIP Status        : problem is solved [optimal solution found]
Solving Time (sec) : 0.00
Solving Nodes      : 0
Primal Bound       : -7.82347272057494e-01 (1 solutions)
Dual Bound         : -7.82347272057494e-01
Gap                : 0.00 %


### Extract and prepare linear inclusion parameters

In [29]:
def _construct_edge_idxs(
    x_coords: NDArray[np.double], y_coords: NDArray[np.double]
) -> NDArray[np.double]:
    """Build two-dimensional vector of x and y coordinates along the edges of input vectors

    The edges will be parallel to x and y axes.
    """
    assert_equal(len(x_coords), len(y_coords))
    return np.vstack(
        (
            np.concatenate(
                (
                    x_coords[x_coords == x_coords.min(initial=np.infty)].repeat(
                        len(y_coords)
                    ),
                    x_coords[
                        np.logical_and(
                            x_coords != x_coords.min(initial=np.infty),
                            x_coords != x_coords.max(initial=-np.infty),
                        )
                    ],
                    x_coords[x_coords == x_coords.max(initial=-np.infty)].repeat(
                        len(y_coords)
                    ),
                    x_coords[
                        np.logical_and(
                            x_coords != x_coords.min(initial=np.infty),
                            x_coords != x_coords.max(initial=-np.infty),
                        )
                    ],
                    x_coords[0:1],
                )
            ),
            np.concatenate(
                (
                    y_coords,
                    np.repeat([y_coords.max(initial=-np.infty)], len(x_coords) - 2),
                    y_coords[-1::-1],
                    np.repeat([y_coords.min(initial=np.infty)], len(x_coords) - 2),
                    y_coords[0:1],
                )
            ),
        )
    )

In [30]:
theta = dict()
theta_0_0 = linear_inclusion.theta_is[0][0][0]
theta_1_0 = linear_inclusion.theta_is[0][1][0]
theta_0 = _construct_edge_idxs(np.array(theta_0_0), np.array(theta_1_0))
theta_0_1 = linear_inclusion.theta_is[1][0][0]
theta_1_1 = linear_inclusion.theta_is[1][1][0]
theta_1 = _construct_edge_idxs(np.array(theta_0_1), np.array(theta_1_1))
z_0_1 = linear_inclusion.z_is[0][0][0]
z_1_1 = linear_inclusion.z_is[0][1][0]
xi_0_1 = linear_inclusion.xi_is[0][0]
xi_1_1 = linear_inclusion.xi_is[0][1]
xi_1 = np.vstack((xi_0_1, xi_1_1))
assert_equal(xi_1.shape, (2, 1))
lower_r_0_1 = linear_inclusion.r_is[0][0][0].inf
lower_r_1_1 = linear_inclusion.r_is[0][1][0].inf
upper_r_i_0_1 = linear_inclusion.r_is[0][0][0].sup
upper_r_i_1_1 = linear_inclusion.r_is[0][1][0].sup
lower_r_1 = np.vstack((lower_r_0_1, lower_r_1_1))
assert_equal(lower_r_1.shape, (2, 1))
upper_r_1 = np.vstack((upper_r_i_0_1, upper_r_i_1_1))
assert_equal(upper_r_1.shape, (2, 1))
points_per_dim = 20
points_per_edges = 4 * (points_per_dim - 1) + 1
x_0 = _construct_edge_idxs(
    np.linspace(theta_0_0.inf, theta_0_0.sup, points_per_dim),
    np.linspace(theta_1_0.inf, theta_1_0.sup, points_per_dim),
)
assert_equal(x_0.shape, (2, points_per_edges))
x_1 = _construct_edge_idxs(
    np.linspace(theta_0_1.inf, theta_0_1.sup, points_per_dim),
    np.linspace(theta_1_1.inf, theta_1_1.sup, points_per_dim),
)
assert_equal(x_1.shape, (2, points_per_edges))

## Compute Linear Inclusion

In [31]:
z_1 = (
    (linear_inclusion.nn_params.weights[0] @ x_0).transpose()
    + linear_inclusion.nn_params.biases[0]
).transpose()
z_1 = np.concatenate((z_1[:, :58], np.flip(z_1[:, 58:-1], axis=1), z_1[:, -1:]), axis=1)
assert z_1.shape == x_0.shape

In [32]:
h_of_z_1 = np.apply_along_axis(linear_inclusion.activation.func, 1, z_1)
assert h_of_z_1.shape == z_1.shape

In [33]:
taylor_approx_1 = linear_inclusion.activation.func(
    xi_1
) + linear_inclusion.activation.deriv(xi_1) * (z_1 - xi_1)
assert taylor_approx_1.shape == z_1.shape

In [34]:
taylor_plus_lower_r_1 = taylor_approx_1 + lower_r_1
assert taylor_plus_lower_r_1.shape == taylor_approx_1.shape
lower_linear_1 = np.array(
    [x - taylor for taylor in taylor_plus_lower_r_1.T for x in x_1.T]
).T
assert_equal(lower_linear_1.shape, (2, np.square(points_per_edges)))

In [35]:
taylor_plus_upper_r_1 = taylor_approx_1 + upper_r_1
assert taylor_plus_upper_r_1.shape == taylor_approx_1.shape
upper_linear_1 = np.array(
    [x - taylor for taylor in taylor_plus_upper_r_1.T for x in x_1.T]
).T
assert lower_linear_1.shape == (2, np.square(points_per_edges))

## Visualize Linear Inclusion with discretizations

In [36]:
fig_linear_inclusion = go.Figure()
fig_linear_inclusion.add_trace(
    go.Scatter(
        x=theta_0[0],
        y=theta_0[1],
        mode="lines",
        name=r"$\Theta^{(0)}$",
        marker_color=color_blind_seque[0],
    )
)
fig_linear_inclusion.add_trace(
    go.Scatter(
        x=z_1[0],
        y=z_1[1],
        mode="none",
        name=r"$z^{(1)}$",
        fill="toself",
        fillcolor=color_blind_seque[3],
    )
)
fig_linear_inclusion.add_trace(
    go.Scatter(
        x=(z_0_1.inf, z_0_1.inf, z_0_1.sup, z_0_1.sup, z_0_1.inf),
        y=(z_1_1.inf, z_1_1.sup, z_1_1.sup, z_1_1.inf, z_1_1.inf),
        mode="lines",
        name=r"$Z^{(1)}$",
        marker_color=color_blind_seque[3],
    )
)
fig_linear_inclusion.add_trace(
    go.Scatter(
        x=lower_linear_1[0],
        y=lower_linear_1[1],
        mode="none",
        name=(
            r"$x^{(1)} - h^{(1)} (\xi^{(1)}) - \frac{\mathrm{d}"
            r"h^{(1)} (\xi^{(1)})}{\mathrm{d} x} (z^{(1)} -"
            r"\xi^{(1)}) - \underline{r}^{(1)}$"
        ),
        fill="toself",
        fillcolor=color_blind_seque[4],
    )
)
fig_linear_inclusion.add_trace(
    go.Scatter(
        x=upper_linear_1[0],
        y=upper_linear_1[1],
        mode="none",
        name=(
            r"$x^{(1)} - h^{(1)} (\xi^{(1)}) - \frac{\mathrm{d}"
            r"h^{(1)} (\xi^{(1)})}{\mathrm{d} x} (z^{(1)} -"
            r"\xi^{(1)}) - \overline{r}^{(1)}$"
        ),
        fill="toself",
        fillcolor=color_blind_seque[2],
    )
)
fig_linear_inclusion.add_trace(
    go.Scatter(
        x=theta_1[0],
        y=theta_1[1],
        mode="lines",
        name=r"$\Theta^{(1)}$",
        marker_color=color_blind_seque[1],
    )
)
fig_linear_inclusion.add_trace(
    go.Scatter(
        x=h_of_z_1[0],
        y=h_of_z_1[1],
        mode="none",
        name=r"$h^{(1)} (z^{(1)})$",
        fill="toself",
        fillcolor=color_blind_seque[1],
    )
)
ytick_vals = np.linspace(z_1[1].min(), z_1[1].max(), 9)
ytick_labels = list(ytick_vals.round(1))
ytick_labels[len(ytick_vals[ytick_vals < xi_1_1]) - 1] = r"$\xi_1^{(1)}$"
ytick_vals[len(ytick_vals[ytick_vals < xi_1_1]) - 1] = xi_1_1
xtick_vals = np.linspace(theta_0[0].min(), theta_0[0].max(), 5)
xtick_vals = np.insert(xtick_vals, len(xtick_vals[xtick_vals < xi_0_1]) + 1, xi_0_1)
xtick_labels = list(xtick_vals.round(1))
xtick_labels[np.where(xtick_vals == xi_0_1)[0][0]] = r"$\xi_0^{(1)}$"
fig_linear_inclusion.update_layout(
    width=660,
    height=590,
    **shared_layout,
    xaxis_title="$x_0^{(i)}$",
    yaxis_title="$x_1^{(i)}$",
    legend=dict(
        yanchor="top",
        y=1.5,
        xanchor="center",
        x=0.5,
        **common_font,
    ),
    **common_tlr_margin_for_layout,
    **common_bottom_axis_title_margins,
)
fig_linear_inclusion.update_xaxes(
    range=[theta_0[0].min(), theta_0[0].max()],
    constrain="domain",
    tickvals=xtick_vals,
    ticktext=xtick_labels,
)
fig_linear_inclusion.update_yaxes(
    **shared_ratio, tickvals=ytick_vals, ticktext=ytick_labels
)
fig_linear_inclusion.show()

In [37]:
linear_inclusion_filename = "linear_inclusion.pdf"
export_plotly_to_pdf(fig_linear_inclusion, linear_inclusion_filename)

linear_inclusion.pdf created


## Plot of linear constraint with z_1 and Z_1 as input

In [38]:
interval_extension_z_1 = _construct_edge_idxs(
    np.linspace(z_0_1.inf, z_0_1.sup, points_per_dim),
    np.linspace(z_1_1.inf, z_1_1.sup, points_per_dim),
)
taylor_approx_interval_extension_z_1 = linear_inclusion.activation.func(
    xi_1
) + linear_inclusion.activation.deriv(xi_1) * (interval_extension_z_1 - xi_1)
assert taylor_approx_interval_extension_z_1.shape == interval_extension_z_1.shape
taylor_interval_plus_lower_r_1 = taylor_approx_interval_extension_z_1 + lower_r_1
assert (
    taylor_interval_plus_lower_r_1.shape == taylor_approx_interval_extension_z_1.shape
)
interval_lower_linear_1 = np.array(
    [x - taylor for taylor in taylor_interval_plus_lower_r_1.T for x in x_1.T]
).T
assert_equal(interval_lower_linear_1.shape, (2, np.square(points_per_edges)))

In [39]:
fig_linear_constraint = go.Figure()
fig_linear_constraint.add_trace(
    go.Scatter(
        x=interval_lower_linear_1[0],
        y=interval_lower_linear_1[1],
        mode="none",
        name=(
            r"$x^{(1)} - h^{(1)} (\xi^{(1)}) - \frac{\mathrm{d}"
            r"h^{(1)} (\xi^{(1)})}{\mathrm{d} x} (Z^{(1)} -"
            r"\xi^{(1)}) - \underline{r}^{(1)}$"
        ),
        fill="toself",
        fillcolor=color_blind_seque[1],
    )
)
fig_linear_constraint.add_trace(
    go.Scatter(
        x=lower_linear_1[0],
        y=lower_linear_1[1],
        mode="none",
        name=(
            r"$x^{(1)} - h^{(1)} (\xi^{(1)}) - \frac{\mathrm{d}"
            r"h^{(1)} (\xi^{(1)})}{\mathrm{d} x} (z^{(1)} -"
            r"\xi^{(1)}) - \underline{r}^{(1)}$"
        ),
        fill="toself",
        fillcolor=color_blind_seque[4],
    )
)
fig_linear_constraint.update_layout(
    width=660,
    height=590,
    **shared_layout,
    xaxis_title="$x_0^{(i)}$",
    yaxis_title="$x_1^{(i)}$",
    legend=dict(
        yanchor="top",
        y=1.5,
        xanchor="center",
        x=0.5,
        **common_font,
    ),
    **common_tlr_margin_for_layout,
    **common_bottom_axis_title_margins,
)
# fig_linear_constraint.update_xaxes(
#     range=[theta_0[0].min(),theta_0[0].max()], constrain="domain"
# )
fig_linear_constraint.update_yaxes(**shared_ratio)
fig_linear_constraint.show()